<h1 style="color:blue">Praktikum 11. </h1>
<h3 style="color:blue">Twitter'i säutsude analüüs. Info eraldamine. Suurte andmetega töötamine. Vikipeedia artiklite töötlus. PhraseTagger ja grammatikad</h3>

Tänases praktikumis tegeleme Twitteri säutsude ja Vikipeedia artiklite analüüsiga ning katsetame automaatset info eraldamist. Viimasega oleme vähesel määral kokku puutunud ka varasemates praktikumides, näiteks leides ajaväljendeid, nimeüksusi, sagedasemaid nimisõnafraase, elusolendeid erinevatest tekstidest jms. Siinses praktikumis käsitleme teemat aga süstemaatilisemalt.

### Info eraldamine

Informatsiooni saab eraldada nii struktureeritud andmetest kui ka vabatekstist. Lähtudes käesoleva aine eesmärkidest tegeleme siin just nimelt tekstiliste andmetega. Vabast (või poolstruktureeritud) tekstist info eraldamisel on eesmärgiks saada sealt kätte info niisugusel moel, et seda oleks võimalik esitada struktureeritud kujul, näiteks tabelina. Leitud infot kasutavad sõltuvalt eesmärkidest kas lõppkasutajad või teised süsteemid (otsingumootorid, andmebaasisüsteemid vms). 

Vabast tekstist info eraldamisega tehti algust juba 1970ndatel, laiemat kõlapinda on see leidnud aga alates 1990ndatest. Koos aina kasvavate (struktureerimata) infohulkadega muutub ka ülesanne aina päevakohasemaks. 

Info eraldamise põhiülesanneteks ongi näiteks nimeolemite tuvastamine, samaviiteliste üksuste tuvastamine ja relatsioonide leidmine. Nimeolemite tuvastamisega tegelesime juba 5. praktikumis: nägime, et eesti keele jaoks on loodud tööriist, mis tuvastab isikunimesid, asukohti ja organisatsioone. Nimeolemeid võib aga käsitleda ka laiemalt, näiteks võib siia alla lugeda ürituste, toodete, haiguste, ravimite vms huvipakkuva tuvastamise.

Samaviiteliste üksuste tuvastamise puhul soovime leida tekstist sõnad/väljendid, mis tähistavad sama nähtust või objekti. Näiteks tekstis *Galaxy S4 teeb paremaid pilte kui nii mõnigi seebikarp. Telefon maksab küll rohkem, ent tal on ka rohkem funktsioone.* tähistavad "Galaxy S4" ja "Telefon" sama objekti. Samaviiteliste üksuste tuvastamise alamülesanne on asesõnade lahendamine. Üldkasutatavaid tööriistu eesti keelele selle jaoks loodud ei ole.

Samuti ei ole üldkasutatavaid tööriistu relatsioonide ehk sõnadevaheliste seoste leidmiseks. 

Info eraldamiseks vabast tekstist kasutatakse nii reeglipõhiseid meetodeid kui ka masinõpet. Reeglipõhiste meetodite puhul kasutatakse käsitsi kirjutatud keelelisi mustreid. Need kipuvad olema väga domeenispetsiifilised: õiges valdkonnas võivad anda häid tulemusi, aga pole kohandatavad teisele valdkonnale. Käsitsi mustrite kirjutamine nõuab loomulikult palju inimtööjõudu, mis on reeglipõhise lähenemise üheks suurimaks miinuseks. Reeglipõhisel lähenemisel võib kasutada erinevaid mustrite kirjutamise viise, nt regulaaravaldisi ja/või grammatikaid. Regulaaravaldistega olete kõik kokku puutunud, grammatikate kirjutamiseks on EstNLTK-s vahendid olemas, millega täna põgusalt tutvust teeme. Grammatikate eeliseks on võimalus moodustada reegleid erinevates märgenduskihtides olevat infot või ka enda defineeritud märgenduskihte kasutades. Mõnevõrra võimalustevaesem, kuid see-eest lihtsam viis fraase eraldada on PhraseTagger. Näiteks, kui meenutate 4. praktikumist nimisõnafraaside eraldamise ülesannet, kus tuli kontrollida järjest asetsevate sõnade sõnaliiki ja vormi, siis seda oleks tegelikult kergem teha PhraseTaggeri abil.

Masinõppepõhised meetodid informatsiooni eraldamisel lähtuvad kas eeldusest, et vaadeldavat ülesannet saab defineerida kui klassifitseerimisprobleemi, või on vaja identifitseerida mingit funktsiooni täitvaid tekstisegmente. Kuna masinõppesse me selles aines ei süvene, siis käsitleme ülesannetes ainult reeglipõhiseid meetodeid. 

### Suurte andmetega töötamisest
Automaatsel infoeraldusel, aga ka enamikul muudel loomuliku keele automaattöötluse ülesannetel, on mõte vaid siis, kui andmeid on palju - rohkem, kui mõistliku ajaga käsitsi läbi jõuaks vaadata. Kui andmeid on piisavalt palju ja/või operatsioonid, mida nendega teha soovime, on piisavalt keerulised, võib aga ka automaattöötlusel ajakulu muutuda kas tüütult või ka teostamatult suureks. Ilmselt olete pidanud selle aine kodutööde lahendamisel mõnigi kord arvuti taga ootama. Kui ootama peab 20 minutit, on see tüütu, aga kui ooteajaks kujuneb näiteks 2 kuud või lausa 200 aastat? 

Seetõttu on oluline pöörata tähelepanu oma koodi efektiivsusele. Kuidas aga aru saada, mis koodis täpselt kaua aega võtab ning kui kaua skript veel jookseb?

#### Mis mu koodis kaua aega võtab? 

Sellele aitab vastust leida *profileerimine*. Pythonis on selleks erinevaid teeke, nt *cProfile*, *line_profiler*. Notebookis on eriti mugav kasutada viimast, ehkki see vajab eraldi installimist (`conda install line_profiler`). *Line_profiler*'ile saab ette anda oma funktsiooni ning ta näitab iga rea kohta, kui kaua selle täitmiseks aega läks: 

In [1]:
# Kasutame Notebook magicut line_profileri laadimiseks
%load_ext line_profiler

In [2]:
# Funktsioon, mida tahame profileerida
def dumb_function(my_list, other_list):
    other_list2 = set(other_list)
    new_thing = []
    for word in my_list:
        for c in word:
            c += 'a'
            
            # Kontrollime esinemist listis
            if 'c' in other_list: 
                x = 1
                
            # Kontrollime esinemist setis    
            if 'c' in other_list2: 
                x = 2
                
            # Kontrollime esinemist setis, 
            # teisendades listi igal sammul uuesti setiks    
            if 'c' in set(other_list): 
                x = 3
                
            new_thing.append(c)    
    return new_thing

In [3]:
# Andmed, millel tahame funktsiooni testida
# (soovitavalt mingi väiksem alamhulk oma andmetest, mille töötlemiseks ei kulu väga kaua)
my_list = ['kana', 'koer', 'vitamiin', 'w4']
other_list = ['ka', 'ke', 'ki', 'aa', 'ee', 'oo', 'ta', 'ia', 'ma', 'ra', 'ka', 'ke', 'ki', 'aa', 'ee', 'oo']

In [4]:
# Jooksutame funktsiooni line_profileriga, et saada iga rea ajakulu
%lprun -f dumb_function dumb_function(my_list, other_list)

Nagu näeme, vaadates veerge *Time*, *Per hit* ja *% Time*, läheb siin kiiremini elemendi esinemise kontrollimine setis, seda aga ainult siis, kui listi setiks teisendame ühe korra, mitte igal iteratsioonisammul uuesti.

#### Kui kaua mu kood veel jookseb?

Kui oleme koodi tööle pannud ning mingi aja oodanud, kuidas teada, kas töö saab varsti valmis või äkki lähebki 200 aastat, nagu enne sai hirmutatud? Lihtne, aga mitte väga ilus variant on tsüklis iga mingi aja tagant printida välja infot selle kohta, kui kaugele jõudnud oleme. Näiteks kasutades *enumerate()* funktsiooni:

In [5]:
for idx, number in enumerate(range(10000000)):
    if idx%1000000 == 0: # prindib välja ainult iga 1 000 000-nda indeksi
        print(idx)
    number2 = number + 1 # siin on see, mida tegelikult tsüklis teha tahame

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000


Selle lähenemise miinuseks on veel see, et peame teadma, kui kiiresti umbes meie kood jooksma hakkab ehk iga mitme iteratsiooni tagant oleks mõistlik printida. **NB!** Kui lasete välja printida nt kõik 10 000 000 indeksit, siis suure tõenäosusega jookseb Notebook lihtsalt kokku. Seega, et mitte koormata arvuti ressursse liialt printimisega, tuleb selleks valida mõistlik intervall. 

Sama funktsiooni täitmiseks on aga olemas ka eraldi teek *tqdm* (vaja installida - `conda install tqdm`), mille kasutamine tsüklis on lihtne, ilus ja mugav:

In [6]:
from tqdm import tqdm

In [7]:
for number in tqdm(range(10000000)):
    number2 = number +1

100%|█████████████████████████████████████████████████████████████████| 10000000/10000000 [00:08<00:00, 1231744.87it/s]


#### Mida teha, et mitte ilmaasjata aega raisata?
Ühest küljest on oluline **koodi optimeerimine** - mitte teha ühte operatsiooni mitu korda, mitte teha aeganõudvaid operatsioone, kui neid saab asendada kiirematega jms. Vähem oluline pole aga ka **üldine töökorraldus**. 

Kui on tegu suuremate andmetega, tasub esmalt koodi **testida mingil väiksel alamulgal andmetest** ja vaadata, kas tulemus on selline, nagu loodetud - pole ju mõtet oodata 20 minutit, parandada koodis näpuviga, oodata jälle 20 minutit, parandada järgmine viga jne. 

Lisaks, kui mingi aeganõudev arvutus on ära tehtud ja tulemus kätte saadud, võib  vahepeal selle **tulemuse salvestada faili** - sel juhul ei pea nt notebooki/arvuti kokkujooksmise, muutujanime kogemata ülekirjutamise vms puhul uuesti otsast peale hakkama. Faili salvestamisel tasub muidugi mõelda sellele, mis kujul salvestada - sobivad näiteks pickle või json, mida on käsitletud eelmistes praktikumides.

### Ülesanne 1. Psüühikahäirete riskigruppi kuuluvate Twitteri-kasutajate tuvastamine (2p)

**NB!** Tegemist on siiski vaid programmeerimisharjutusega, mille tulemuste põhjal ei tasu kindlasti tegelikult ühtegi Twitteri-kasutajat sildistama hakata.

Laias maailmas on tehtud mitmeid uuringuid selle kohta, kuivõrd on võimalik kasutaja sotsiaalmeediapostituste alusel tuvastada seda, kas ta kuulub psüühikahäirete (depressioon, bipolaarne häire, jms) riskigruppi. On leitud mitmeid nii keelelisi kui ka mittekeelelisi tunnuseid, mille abil niisuguseid kasutajaid tuvastada. Siin ülesandes proovime, kas ka eestikeelsete säutsude põhjal oleks võimalik midagi sarnast teha. Selleks on siin pakitud kaustas "tweets" ~175 000 eestikeelset Twitteri säutsu.

**NB!** Kuna säutse on palju, siis võib nende töötlemine võtta aega, eriti, kui kasutatav arvuti aeglasepoolne on. Kui tundub, et oma kood on mõistlikult kirjutatud ning näib, et kõigi säutsude töötlemisele kuluks üle 30 minuti (selle ennustuse tegemiseks ei pea 30 min ära ootama - vt ülevaltpoolt juhiseid), siis võib kasutada omal valikul ka mingit väiksemat hulka säutsudest.

Võtame eeskujuks [selles](https://www.aclweb.org/anthology/W14-3207.pdf) artiklis väljatoodud tunnused, mida on kasutanud mitmed autorid ja mis inglise keele peal usaldusväärseid tulemusi on andnud. Need keskenduvad psüühikahäiretega inimestele omastele sümptomitele nagu negatiivsus, vähene sotsiaalne kaasatus jne. Lähtudes eesti keele võimalustest ja vajadustest, kasutame siin ülesandes järgmisi tunnuseid:
* postituse kellaaeg - psüühikahäiretega inimesi vaevab sageli unetus ja seetõttu postitavad nad tihti öösiti
* teiste kasutajatega suhtlemine - psüühikahäiretega inimesed on sageli vähem sotsiaalselt kaasatud, Twitteris toimuvad aga n-ö vestlused just teise kasutaja kasutajanime mainimise teel
* Esimese isiku sage kasutus - viidatud artiklis on leitud, et psüühikahäiretega inimesed kasutavad ingliskeelsetes postitustes rohkem 1. isiku asesõnu. Kuna eesti keeles võib isikut väljendada ka verbivormis, siis vaatleme nii 1. isikus asesõnu kui ka vastavaid verbivorme (kõigis aegades) 
* negatiivsete sõnade sage kasutus - kasutame EKI emotsioonileksikonis toodud negatiivseid sõnu

Seega, püüame leida igale kasutajale nende tunnuste põhjal riskigruppi kuulumise skoori. Kui järjestame kasutajad skooride alusel, võime eeldada, et suurema skoori saanud kasutajad kuuluvad psüühikahäirete riskigruppi suurema tõenäosusega. 

**Andmed:**
* säutsud - siin. Kui laadite alla ja pakite kausta lahti, leiate json-formaadis failid
* emotsioonileksikon - fail *estonian-emotion-lexicon.csv*. Fail on Windowsi kodeeringus ('1257') ning eraldajaks ('delimiter') on semikoolon (;). Kuna kirjanduses ei ole leitud seost positiivsete sõnade ja psüühikahäirete esinemise/mitteesinemise vahel, siis kasutame failist ainult negatiivseid (miinusmärgiga skooriga) sõnu. **NB!** Failis on sõnavormid, mitte lemmad.

**Skoori arvutamine:**
* Arvutame igale kasutajale eraldi skoorid unetuse, negatiivsuse, 1. isiku kasutuse ning teiste kasutajate mainimise kohta. **Lõpuks liidame 4 skoori kokku ning selle alusel järjestame tulemused.**
* **Unetus:** iga postitus, mis on tehtud ajavahemikus 01:00 - 06:00 öösel, lisab skoorile ühe punkti. Jagada läbi kasutaja postituste arvuga. (St: kui kasutaja on teinud 20 postitust, millest 5 on tehtud vahemikus 01:00-06:00, siis on kasutaja unetuse skoor 5/20 = 0,4). Vaatame andmetest tunnust 'created_at'
* **Sotsiaalne kaasatus:** iga vastus teisele kasutajale lisab skoorile ühe punkti. Jagada läbi kasutaja postituste arvuga. Vaatame andmetest tunnust 'in_reply_to_screen_name'. **NB!** Kuna sotsiaalne kaasatus on pöördtunnus - kõrgem väärtus tähendab väiksemat psüühikahäirete riski - , siis tuleks selle lõppskoorile lisada ette miinusmärk.
* **1. isiku kasutus:** iga 1. isikus asesõna või verb lisab skoorile ühe punkti. Jagada läbi kasutaja postituste arvuga. Vaatame andmetest tunnust 'text'
* **Negatiivsus:** iga leksikonis negatiivset skoori omav sõna postituses lisab skoorile nii palju punkte nagu emotsioonileksikonis skoorina antud on. Kuna soovime lõpus kõik skoorid kokku liita, tuleks siinkohal ka miinusmärgiga arvude asemel kasutada absoluutväärtusi. Jagada läbi kasutaja postituste arvuga. Vaatame andmetest tunnust 'text'

Vaadake peale paari kõige kõrgema ja kõige madalama skoori saanud kasutaja säutsudele. Kas tundub, et metoodikat oleks võimalik tulemuslikult rakendada ka eesti keele peal?

### Vikipeedia töötlus EstNLTK 1.4-s

Kui soovime saada palju andmeid erinevate teemade kohta, siis üheks kasulikuks allikaks on kindlasti Vikipeedia. EstNLTK uues versioonis pole veel vahendeid eestikeelse Vikipeedia töötlemiseks, küll aga võimaldab seda EstNLTK 1.4. Seal on olemas töövahendid eestikeelse Vikipeedia allalaadimiseks ning xml-kujult json-kujule viimiseks (juhised [siin](https://estnltk.github.io/estnltk/1.4.1/tutorials/wikipedia.html#extracting-articles-from-xml-files)). 
Selleks on aga esmalt tarvis luua uus conda keskkond, mis kasutab Python 3.5 versiooni ning sellele installida EstNLTK 1.4. 
Tuleb aga arvestada, et eestikeelne Vikipeedia on päris suur ja seega kogu Vikipeedia töötlemine võtab kaua aega ("kaua" tähendab siinkohal seda, et sõltuvalt arvutist läheb mitmeid tunde või terve päev). 

**Siinses praktikumis pole vaja eelmist EstNLTKd installida ja Vikipeediat alla laadida, kasutame töötlemiseks juba eelnevalt kogutud ja json-kujule viidud väikest alamhulka Vikipeedia failidest.**

### Ülesanne 2. Isikute ja asukohtadega seotud informatsiooni ekstraheerimine tekstist (2,5p)

Oletame, et soovime luua andmebaasi selle kohta, millistes kohtades mingi kunstnik viibis ja milliseid stiile viljeles. Andmebaasi põhjal oleks võimalik lisaks konkreetsete kunstnike andmetele leida infot, missuguses asukohas mingit kunstistiili kõige enam viljeleti. 

Seega, **eesmärk** on ekstraheerida kunstnike Vikipeedia artiklitest informatsiooni nende:
   * seotusest erinevate asukohtadega - kes kunstnikest millistes asukohtades (riigid, linnad, külad jne) sündis, elas, töötas, maalis jne.
   * viljeletud kunstisuundadest
   
Selle saavutamiseks kasutame EstNLTK vahendeid kombineeritult reeglipõhise lähenemisega.

**Andmed:** 
* Tekstid on pakituna kaustas *kunstnikud*. Kui kausta lahti pakite, leiate ~1500 json-formaadis Vikipeedia artiklit.
* Kunstistiilide leksikon on failis *kunstivoolud.txt*. Enamiku kunstivoolude kohta leidub leksikonis enam kui 1 märksõna, mida tuleb väljundis käsitleda võrdsetena (impressionism = impressionist = impressionistlik).

**Loodava programmi abil:**
* loetakse etteantud kaustast sisse failides olevas tekstid,
* tuvastatakse tekstidest sellised osalaused, milles sisaldub:
     - vähemalt üks asukohale viitav nimega üksus (LOC) 
     - JA/VÕI
     - vähemalt üks kunstivoolule viitav märksõna (failist "kunstivoolud.txt")
     
**NB! Osalausete läbikäimisel tuleta meelde 9. praktikumi**
     
* kui tuvastatud osalauses leidub ka pärisnimele viitavaid nimega üksusi (PER), siis tuleks kontrollida, kas vähemalt mõnel neist on midagi ühist failinimes oleva kunstniku nimega (ees- või perekonnanimi - võime oletada, et kui Aado Vabbe failis räägitakse Aadost, siis on tegu ikka selle sama kunstnikuga, aga kui Viiraltist, siis ilmselt mitte). Kui tundub, et osalause räägib ainult mõnest teisest isikust, siis ärme sellest osalausest leitud asukohale ja/või kunstistiilidele tähelepanu pööra.
* kui tuvastatud osalauses pärisnimele viitavaid üksusi üldse ei leidu, siis omistame leitud asukohad ja kunstivoolud kunstnikule, kelle nime leiame faili nimest
* väljundiks peaks olema kaks json-formaadis faili - üks sisaldab iga kunstniku asukohti, teine iga kunstniku viljeldud kunstistiile. Kunstnike nimed on sellisel kujul nagu failinimedes.

**NB2! Nimeolemite eraldamisel tuleb siingi arvestada, et töö võib olla ajamahukas**

Millised asukohad on impressionistide seas kõige populaarsemad?

### Fraaside eraldamine

Eelnevates ülesannetes eraldasime tekstidest informatsiooni üksteisest eraldiseisvaid üksusi vaadates – leidsime asesõnu, nimeüksusi, võrdlesime tekstisõnu etteantud leksikoniga. Mõnikord nõuab ülesanne aga üldisemat lähenemist, näiteks on vaja lähtuda teksti struktuurist. Kui tahame eraldada järjestikkuseid sõnu/tekstiosi hõlmavaid struktuure, saame seda teha **`PhraseTagger`**'i abil. PhraseTagger võimaldab kirjeldada mõne kihi atribuutidest koosnevaid mustreid ning märgendada tekstist neile vastavaid fraase. 

Vaatame näiteks, kuidas saaks tekstist eraldada lihtsaid [komparatiivtarindeid](https://et.wikipedia.org/wiki/Adjektiivne_v%C3%B5rdlustarind#Komparatiivtarind), täpsemalt juhtusid, kus omadussõna on keskvõrdes ning sellele järgneb sidesõna *kui* abil seotud võrdlusalus. Märgendame kihi, mida sisendina kasutada

In [8]:
from estnltk import Text

text = Text("Andres oli alguses pikem kui Jaan. "
           " Nüüdseks on Jaan kasvanud isegi pikemaks kui Andresestki pikem Taavi. "
           " Villu oli märksa lühem kui teised poisid ja ainult natuke suurekasvulisem kui õde Ann."
           " Selles vanuses pole enam pikemaks aga Villul lootust kasvada.")

text.tag_layer(['morph_analysis'])

Text(text='Andres oli alguses pikem kui Jaan.  Nüüdseks on Jaan kasvanud isegi pikemaks kui Andresestki pikem Taavi.  Villu oli märksa lühem kui teised poisid ja ainult natuke suurekasvulisem kui õde Ann. Selles vanuses pole enam pikemaks aga Villul lootust kasvada.')

Loome PhraseTaggeri, mis tekitab kihi kindlas järjestuses sõnaliikide alusel. Vajalikud sõnaliikide järjestused anname taggerile sõnastike listina atribuudis `vocabulary`.

In [9]:
from estnltk.taggers import PhraseTagger

phrase_list = [{'phrase': ('C', 'J', 'S')},
              {'phrase': ('C', 'J', 'P')},
              {'phrase': ('C', 'J', 'H')},
              {'phrase': ('H', 'C')},
              {'phrase': ('P', 'C')},
              {'phrase': ('S', 'C')}]

phrase_tagger = PhraseTagger(output_layer='comparative_phrases',
                             input_layer='morph_analysis',
                             input_attribute='partofspeech',
                             vocabulary=phrase_list,
                             key='phrase',
                             output_attributes=['phrase'])

phrase_tagger.tag(text)

Text(text='Andres oli alguses pikem kui Jaan.  Nüüdseks on Jaan kasvanud isegi pikemaks kui Andresestki pikem Taavi.  Villu oli märksa lühem kui teised poisid ja ainult natuke suurekasvulisem kui õde Ann. Selles vanuses pole enam pikemaks aga Villul lootust kasvada.')

PhraseTaggeri loomisel olulisemad atribuudid:

`output_layer` - mis kihti leitavad fraasid lisame


`input_layer` - sisendkiht, mille põhjal fraase leiame


`input_attribute` - sisendkihi atribuut, mis on fraaside leidmise aluseks


`vocabulary` - selle parameetriga saab ette anda nt (csv-)failinime, listi või Vocabulary-objekti, mis sisaldab mustreid, mida leida tahame, ja mille tagger võtab fraaside määramise aluseks


`key` - vocabulary element, mis määrab leitava mustri


`output_attributes` - list atribuutidest, mis tuleb väljundkihis esitada


`decorator` - kahe argumendiga (span ja annotation) funktsioon, mis tagastab True, kui märgendus on korrektne, muul juhul False. Dekoraatori sees saab seada tingimusi, millele annotation peab vastama ja vajadusel saab sellele ka uusi atribuute lisada.

`conflict_resolving_strategy` - konfliktilahendusstrateegia


`priority_attribute` - *vocabulary*'s defineeritud prioriteeti tähistava atribuudi nimi


Ülejäänud atribuutide kohta saab rohkem informatsiooni [siit](https://github.com/estnltk/estnltk/blob/version_1.6/tutorials/taggers/phrase_tagger.ipynb).

Vaatame nüüd saadud kihti.

In [10]:
text.comparative_phrases

Layer(name='comparative_phrases', attributes=('phrase',), spans=SL[EnvelopingSpan(['pikem', 'kui', 'Jaan'], [{'phrase': ('C', 'J', 'H')}]),
EnvelopingSpan(['pikemaks', 'kui', 'Andresestki'], [{'phrase': ('C', 'J', 'H')}]),
EnvelopingSpan(['lühem', 'kui', 'teised'], [{'phrase': ('C', 'J', 'P')}]),
EnvelopingSpan(['suurekasvulisem', 'kui', 'õde'], [{'phrase': ('C', 'J', 'S')}]),
EnvelopingSpan(['pikemaks', 'aga', 'Villul'], [{'phrase': ('C', 'J', 'H')}])])

Väljundist torkab silma kaks puudust: 

1) tuleb välja, et lisaks *kui*-tarindile võime tulemusse saada ka teiste, soovimatute sidesõnadega seotud fraase. Fraaside filtreerimiseks saab dekoraatorisse lisada tingimusi, mille põhjal fraase aktsepteerida või välja jätta. Samuti saab määrata kihile uusi atribuute ja nende väärtusi. 

Lahendame selle probleemi, luues dekoraatori, mis aktsepteerib fraase, mille sidesõnaks on *kui*, ja mustrile ('H', 'C') vastavaid fraase, mille puhul pärisnimi on seestütlevas käändes. Lisame kihile atribuudi, mis sisaldab võrdlusaluseks olevat tekstisõna.

In [11]:
def decorator(span, annotation):
    if len(span) == 2:
        for anno in span[0].annotations:
            # annotation.phrase kaudu saame kätte vaatlusaluse mustri
            if anno.partofspeech == annotation.phrase[0] and 'el' in anno.form:
                annotation['base_of_comparison'] = span[0].text
                return True
    
    if span[1].text == 'kui':
        annotation['base_of_comparison'] = span[2].text
        return True
    
    return False

2) Teiseks on puudu järjestusele ('H', 'C') vastav fraas, kuigi tekstis vastab sellele *Andresestki pikem*. Probleem tekib lõigust *pikemaks kui Andresestki pikem*. Nagu näha, on siinkohal kattuvus mustrite ('C', 'J', 'H') ning ('H', 'C') vahel. Vaikimisi jätab PhraseTagger kattuvuste korral alles vaid pikema järjendi. Seda saame aga muuta.

PhraseTagger kasutab kattuvate fraaside lahendamiseks kihi meetodit `resolve_conflicts`. Sellele saame kattuvustega tegelemiseks ette anda konfliktilahendusstrateegia ja soovi korral ka mustrite prioriteedid. Parameetriga `conflict_resolving_strategy` on võimalik kattuvusi lahendada järgmisi väärtusi ette andes:
  - *ALL* - alles jäävad kõik vasted, võttes arvesse seatud prioriteete;
  - *MAX* - alles jääb maksimaalse pikkusega vaste, võrdsete pikkuste puhul kõrgema prioriteediga vaste;
  - *MIN* - alles jääb minimaalse pikkusega vaste, võrdsete pikkuste puhul kõrgema prioriteediga vaste.

Seega, kui tahaksime, et loetletud oleksid kõik vasted, saame taggerit luues konfliktilahendusstrateegia ette anda, lisades `conflict_resolving_strategy='ALL'`, kuid arvesse peab võtma, et kattuvate Span'ide korral ei saaks me nt kasutada `layer.display()` meetodit kihi visualiseerimiseks. 

Oletame, et meil on vaja *pikemaks kui Andresestki pikem* lõigust kätte saada hoopis fraas *Andresestki pikem*. Kui me oleme täiesti kindlad, et esmajärjekorras on meil alati vaja kätte saada lühem vaste, saame valida konfliktilahendusstrateegia 'MIN'. 

Samas võib mõnikord tekkida olukordi, kui konflikt tekib kahe sama pika järjendi vahel või mõnel juhul tahame alles jätta pikemat, aga teisel juhul lühemat fraasi. Seesuguste olukordade lahendamiseks saame fraasidele määrata prioriteedid. PhraseTaggerit luues paneme need kirja *vocabulary*'sse: prioriteetsem fraas saab väiksema arvulise väärtuse. Kindlasti tuleb prioriteete kasutades määrata märgendajat luues ka atribuut `priority_attribute`, mis viitab, millise kihiatribuudi alt prioriteedi leiame.   

Lisame nüüd *vocabulary*'sse prioriteedid ja katsetame konfliktilahendust, mis arvestab ainult etteantud prioriteete (st strateegiaks peab olema 'ALL'). Määrame ka dekoraatori:

In [12]:
# Loome uue taggeri, mis sisaldab dekoraatorit ja konfliktilahendust
# output_attributes peab sisaldama ka dekoraatoris määratud atribuute

phrase_tagger = PhraseTagger(output_layer='comparative_phrases',
                             input_layer='morph_analysis',
                             input_attribute='partofspeech',
                            vocabulary=[{'phrase': ('C', 'J', 'S'), 'priority': 2},
                                       {'phrase': ('C', 'J', 'P'), 'priority': 2},
                                       {'phrase': ('C', 'J', 'H'), 'priority': 2},
                                       {'phrase': ('H', 'C'), 'priority': 1},
                                       {'phrase': ('P', 'C'), 'priority': 1},
                                       {'phrase': ('S', 'C'), 'priority': 1}],
                            key='phrase',
                            decorator=decorator,
                            conflict_resolving_strategy='ALL',
                            priority_attribute='priority',
                            output_attributes=['phrase', 'base_of_comparison'])


In [13]:
text.pop_layer('comparative_phrases')  # eemaldame eelmise taggeriga saadud kihi

Layer(name='comparative_phrases', attributes=('phrase',), spans=SL[EnvelopingSpan(['pikem', 'kui', 'Jaan'], [{'phrase': ('C', 'J', 'H')}]),
EnvelopingSpan(['pikemaks', 'kui', 'Andresestki'], [{'phrase': ('C', 'J', 'H')}]),
EnvelopingSpan(['lühem', 'kui', 'teised'], [{'phrase': ('C', 'J', 'P')}]),
EnvelopingSpan(['suurekasvulisem', 'kui', 'õde'], [{'phrase': ('C', 'J', 'S')}]),
EnvelopingSpan(['pikemaks', 'aga', 'Villul'], [{'phrase': ('C', 'J', 'H')}])])

In [14]:
# Märgendame kihi uuendatud taggeriga
phrase_tagger.tag(text)
text.comparative_phrases

Layer(name='comparative_phrases', attributes=('phrase', 'base_of_comparison', 'priority'), spans=SL[EnvelopingSpan(['pikem', 'kui', 'Jaan'], [{'phrase': ('C', 'J', 'H'), 'base_of_comparison': 'Jaan', 'priority': 2}]),
EnvelopingSpan(['Andresestki', 'pikem'], [{'phrase': ('H', 'C'), 'base_of_comparison': 'Andresestki', 'priority': 1}]),
EnvelopingSpan(['lühem', 'kui', 'teised'], [{'phrase': ('C', 'J', 'P'), 'base_of_comparison': 'teised', 'priority': 2}]),
EnvelopingSpan(['suurekasvulisem', 'kui', 'õde'], [{'phrase': ('C', 'J', 'S'), 'base_of_comparison': 'õde', 'priority': 2}])])

### Ülesanne 3. Nimisõnafraaside eraldamine (1p + 0,5 boonuspunkti)

Püüdke kirjutada PhraseTagger, mis märgendab tekstis n-ö laiemas tähenduses nimisõnafraase: erinevalt keeleteadusest, ei ole info eraldamise seisukohast üldiselt vahet, kas tekstis on öeldud "poes on hea teenindus" või "teenindus oli poes hea" või hoopis "poes teenindus hea". Katsetage taggerit faili *hinnavaatlus.csv* tekstide peal. Märgenduse peaksid saama minimaalselt järgmised nädisstruktuurid:

* suur õun
* suur ja sinine õun
* suur sinine õun
* õun on suur
* õun suur
* õun on suur ja sinine

Seejuures sõnad 'õun', 'suur' ja 'sinine' peaksid olema üldistatud.

Märgendatavale kihile lisage ka atribuut, mis sisaldab järjendit normaliseeritud fraasist: viige leitud fraasid kujule "suur õun" e omadussõna, millele järgneb nimisõna. Kui leidub mitu omadussõna, tehke igast eraldi fraas ("suur ja sinine õun" -> ["suur õun", "sinine õun"]), kusjuures fraas peaks olema algvormis. Mitmeste analüüside puhul jälgige, et algvorm oleks korrektne ("tehtud töö", mitte "tegema töö").

**0,5 boonuspunkti** saab teenida fraaside täpsema filtreerimisega. Uuri väljundist, milliseid ebatäpsusi näed ning paranda taggerit nii, et see likvideeriks mõne vigadest (kirjelda, milliseid vigu parandasid). Vihje: alustuseks võiks mõelda näiteks, mis on (süsteemselt) valesti lausetes *Olen Tartu **esindustest ostnud** probleemivabalt* või *Õnneks sai mul üks **tuttav garantiiga** aidata*.

### Lisamaterjal: Grammatikad EstNLTK-s

PhraseTaggerist keerukam, kuid mõnevõrra võimalusterohkem on kontekstivaba grammatika kirjutamine. Selle jaoks on EstNLTK-s klassid Grammar ja GrammarParsingTagger.

Grammatikate puhul defineerime ära n-ö ehitusplokid, millest meie soovitavad üksused koosnevad, ning reeglid, kuidas plokid omavahel kombineeruvad. Loodud grammatika anname ette GrammarParsingTaggerile, mis märgendab teksti grammatikast lähtudes.

Loome näiteks grammatika, mis märgendab primitiivseid omadussõnafraase (selle ülesande jaoks on olemas ka spetsiaalne [AdjectivePhraseTagger](https://github.com/estnltk/estnltk/blob/version_1.6/tutorials/taggers/adjective_phrase_tagger.ipynb), kuid õppimise mõttes proovime ise üht luua).

Tahame märgendada järgmisi konstruktsioone:
 - suur
 - suur ja ilus
 - ilus suur
 - suurem
 - suurem ja ilusam
 
Alustuseks loome Grammar-objekti:

In [15]:
from estnltk.finite_grammar import Grammar, Rule
from estnltk.taggers import GrammarParsingTagger

In [16]:
grammar = Grammar(start_symbols=['ADJECTIVE_CHAIN'],
                  depth_limit=float('inf'), # vaikimisi
                  width_limit=float('inf'), # vaikimisi
                  legal_attributes=['comparison_degree', 'pattern']
                  )

`Start_symbol`-iga määrame, millisele reeglile vastavaid järjendeid tahame väljundisse. Kasutades üht algussümbolit, saame lihtsama ja paremini hoomatava mudeli.
Parameetri `legal_attributes` alla tuleb märkida dekoraatoriga lisatavad atribuudid (sellest tuleb peagi juttu).

Nüüd on vaja grammatikasse reegleid lisada. Reegel koosneb vasakust ja paremast poolest, kus vasakul pool on üks mitteterminaal ja paremal seda määravad terminaalid (siinse näite puhul sõnaliigimärgendid) või mitteterminaalid. Pooled on üksteisega asendatavad. Erinevalt PhraseTaggerist saab grammatikareeglites teha rohkem üldistusi. Näiteks kui oleme reeglitega juba määranud, et omadussõnana käsitleme ükskõik millist omadussõna võrdlusastet, ei pea me fraasi defineerimiseks enam kirjutama eraldi reegleid nii algvõrdes kui ka keskvõrdes omadussõnale.

Selleks, et täpsustada, millised reeglitele leitavad vasted väljundisse sobivad, saame igale reeglile määrata validaatori. Samuti on võimalik reeglile lisada dekoraator, mis võimaldab väljundkihi atribuutite väärtustada.

Kirjutame reeglid, mis käsitlevad nii alg- kui ka keskvõrret ühtmoodi ning aktsepteerivad määrsõna nende ees. Lisame reeglitele dekoraatori, mis lisab kihiatribuudi alla võrdlusastme (esimese omadussõna järgi) ning mustri, mille järgi vaste leitud on. Reeglile, mis eraldab sidesõnaga seotud omadussõnu, lisame validaatori, mis kontrollib, kas omadussõnad ühilduvad vormis:

In [17]:
def decorator(nodes):
    degree = ''
    
    # Muster, mille põhjal vaste on leitud
    pattern = ' '.join([node.name for node in nodes])
    
    # Lisame mustri esimesele elemendile vastava terminaali järgi võrdlusastme
    if nodes[0].terminals[0].name == 'A':
        degree = 'positive'
    elif nodes[0].terminals[0].name == 'C':
        degree = 'comparative'

    return {'comparison_degree': degree,
            'pattern': pattern}

def validator(nodes):
    adj1 = nodes[0].terminals[0].attributes['form']  # terminals alt saame kätte vajaliku vormi
    adj2 = nodes[2].terminals[0].attributes['form']
    if adj1 == adj2:
        return True
    return False

In [18]:
# ADJECTIVE võib tähistada määramatut hulka järjestikusi algvõrdes omadussõnu
grammar.add_rule('ADJECTIVE', 'MSEQ(A)', group='g0', decorator=decorator)  

# ADJECTIVE võib tähistada määramatut hulka järjestikusi keskvõrdes omadussõnu
grammar.add_rule('ADJECTIVE', 'MSEQ(C)', group='g0', decorator=decorator)

# ADJECTIVE_CHAIN võib olla ADJECTIVE või kaks sidesõnaga seotud ADJECTIVE'i
grammar.add_rule('ADJECTIVE_CHAIN', 'ADJECTIVE', group='g0', priority=3, decorator=decorator)
grammar.add_rule('ADJECTIVE_CHAIN', 'ADJECTIVE J ADJECTIVE', group='g0', priority=1, validator=validator, decorator=decorator)

Kui teame, et meie mustris võib olla (teadmata arv) korduvaid osi, saame seda edasi anda lisades selle MSEQ- või SEQ-reeglite abil. MSEQ-reegliga saab määrata korduvad tekstiosad, nii et tagastatakse pikim võimalik variant, samamoodi kasutatav SEQ-reegel tagastab kõik võimalikud variandid. Kuna fraasides võib esineda erinimelisi täiendeid, mida ei eraldata komaga, ja meid huvitavad kõik neist, saamegi kirjutada reegli MSEQ(A), mis tuvastab _**suur punane** õhupall_ omadussõnad ühe üksusena.
Dekoraatori kasutamisel tuleb vajadusel MSEQ- ja SEQ-reegleid eksplitsiitselt käsitleda.

<div class="alert alert-block alert-warning">
<b>Vahemärkus:</b> MSEQ tekitab eraldi grupi, mis juskui ei konkureeri tavaliste prioriteetide alusel, tekitades väljundisse soovimatuid vasteid. Kui see on oluline, tuleb dekoraatoris prioriteedid üle kirjutada ja hiljem kihi kattuvused <i>resolve_conflicts</i>-funktsiooniga lahendada. Vt näidet altpoolt.
</div>

Kuna meie jaoks ei ole oluline, mis võrdlusastmes omadussõnaga tegu on, saime kirjutada üldistava reegli, millega määrame nii algvõrded (A) kui ka keskvõrded (C) mitteterminaali 'ADJECTIVE' alla.
'ADJECTIVE_CHAIN' reeglite vasted on need, mis jõuavad väljundisse.

In [19]:
grammar


Grammar:
	start: ADJECTIVE_CHAIN
	terminals: J, MSEQ(A), MSEQ(C)
	nonterminals: ADJECTIVE, ADJECTIVE_CHAIN
	legal attributes: frozenset({'pattern', 'comparison_degree'})
	depth_limit: inf
	width_limit: inf
Rules:
	ADJECTIVE -> MSEQ(A)	: 0, val: default_validator, dec: decorator, scoring: default_scoring
	ADJECTIVE -> MSEQ(C)	: 0, val: default_validator, dec: decorator, scoring: default_scoring
	ADJECTIVE_CHAIN -> ADJECTIVE	: 3, val: default_validator, dec: decorator, scoring: default_scoring
	ADJECTIVE_CHAIN -> ADJECTIVE J ADJECTIVE	: 1, val: validator, dec: decorator, scoring: default_scoring

Nüüd, kui grammatika on tehtud, peame tekstile vastava märgenduse lisamiseks tekitama GrammarParsingTaggeri. Loomisel peame määrama kasutatava grammatika. Samuti on vaja määrata, mis nimega kihti ja millist selle atribuuti sisendina kasutame. Muidugi kirjeldame ka väljundkihi ja -atribuudid.

In [20]:
grammar_tagger = GrammarParsingTagger(grammar=grammar,
                                      name_attribute='partofspeech', # vaikimisi 'grammar_symbol'
                                      layer_of_tokens='morph_analysis',
                                      attributes=('comparison_degree', 'pattern'),
                                      output_layer='adjectives',
                                      output_ambiguous=True)

Vaatame, kuidas grammatika tekstil töötab:

In [21]:
text = Text('Eriti isuäratavad ja maitsvad saavad koogid antonovkast.'
' Kõige magusamad õunasordid jätkem lihtsalt maiustamiseks.'
' Nendest ei pruugi saada nii häid mahlaseid kooke kui hapukatest õuntest.').tag_layer(['morph_analysis'])

In [22]:
grammar_tagger.tag(text)
text.adjectives

Layer(name='adjectives', attributes=('comparison_degree', 'pattern'), spans=SL[EnvelopingSpan(['isuäratavad', 'ja', 'maitsvad', 'saavad'], [{'comparison_degree': 'positive', 'pattern': 'ADJECTIVE J ADJECTIVE'}]),
EnvelopingSpan(['magusamad'], [{'comparison_degree': 'comparative', 'pattern': 'ADJECTIVE'}]),
EnvelopingSpan(['häid', 'mahlaseid'], [{'comparison_degree': 'positive', 'pattern': 'ADJECTIVE'}]),
EnvelopingSpan(['hapukatest'], [{'comparison_degree': 'positive', 'pattern': 'ADJECTIVE'}])])

Praegusel juhul vaatlesime väga lihtsakoelise grammatika loomist, tuginedes *morph_analysis* kihile. Kui aga oskame ise Taggereid luua, nagu 9. praktikumis õppisime, saame oluliselt laiendada ka grammatikate kasutamist.
Rohkem informatsiooni grammatikate loomise ja kasutamise kohta leiad [siit](https://github.com/estnltk/estnltk/blob/version_1.6/tutorials/finite_grammar/introduction_to_finite_grammar.ipynb)

---

### MSEQ-prioriteedi seadmine

Eespool leidus väide, et MSEQ-reegel tekitab eraldi grupi, mis võib teatud juhtudel, hoolimata reegli prioriteetidest, tekitada väljundisse soovimatuid vasteid. Järgnevalt vaatame ühte sellise probleemi tekkimist ja lahendamist.

Kirjutame veel lihtsama grammatika kui enne: meid huvitavad vaid algvormis omadussõnad, mis võivad esineda ühekaupa, mitmekaupa järjest või sidesõnaga eraldatult kahekaupa. Seame prioriteedid nii, et kattuvuste korral jääks alles sidesõnaga ühendatud omadussõnad.

In [23]:
grammar_2 = Grammar(start_symbols=['ADJECTIVE_CHAIN'])
grammar_2.add_rule('ADJECTIVE_CHAIN', 'A J A', group='g0', priority=3)
grammar_2.add_rule('ADJECTIVE_CHAIN', 'MSEQ(A)', group='g0', priority=1)

In [24]:
grammar_tagger = GrammarParsingTagger(grammar=grammar_2,
                                      name_attribute='partofspeech',
                                      layer_of_tokens='morph_analysis',
                                      output_layer='adjectives_2',
                                      output_ambiguous=True)

In [25]:
grammar_tagger.tag(text)
text.adjectives_2

Layer(name='adjectives_2', attributes=(), spans=SL[EnvelopingSpan(['isuäratavad'], [{}]),
EnvelopingSpan(['isuäratavad', 'ja', 'maitsvad'], [{}]),
EnvelopingSpan(['maitsvad', 'saavad'], [{}]),
EnvelopingSpan(['häid', 'mahlaseid'], [{}]),
EnvelopingSpan(['hapukatest'], [{}])])

Näeme, et MSEQ(A) reegli vasted on aktsepteeritud ka juhtudel, kui prioriteedi alusel oleks pidanud eelistama teist reeglit, st järjendid ['isuäratavad'] ja ['maitsvad', 'saavad'] on üleliigsed. Sellel puhul pole teha muud, kui lisada MSEQ-reeglile parameeter `scoring`, mis peab olema suurem kui 0, et kõik vastavused jõuaksid kindlasti väljundisse. Seejärel tuleb dekoraatoris prioriteedid atribuutidena üle määrata ning viimaks `resolve_conflicts`-meetodiga soovimatud märgendused eemaldada. Vaatame, kuidas see käib:

In [26]:
def decorator(nodes):
    degree = ''
    pattern = ' '.join([node.name for node in nodes])
    
    #  Esiteks määrame dekoraatoris taaskord prioriteedid:
    priority = 0
    if pattern == 'MSEQ(A)':
        priority = 2
    elif pattern == 'A J A':
        priority = 1

    return {'pattern': pattern,
            'priority': priority}

In [27]:
grammar_2 = Grammar(start_symbols=['ADJECTIVE_CHAIN'], legal_attributes=['priority', 'pattern'])
grammar_2.add_rule('ADJECTIVE_CHAIN', 'A J A', group='g0', priority=1, decorator=decorator)

# Lisame MSEQ-reeglile parameetri "scoring"
grammar_2.add_rule('ADJECTIVE_CHAIN', 'MSEQ(A)', group='g0', priority=2, decorator=decorator, scoring=lambda node: 1)

In [28]:
grammar_tagger = GrammarParsingTagger(grammar=grammar_2,
                                      name_attribute='partofspeech',
                                      layer_of_tokens='morph_analysis',
                                      attributes=('pattern', 'priority'),
                                      output_layer='adjectives_3',
                                      output_ambiguous=False)

In [29]:
grammar_tagger.tag(text)
text.adjectives_3

Layer(name='adjectives_3', attributes=('pattern', 'priority'), spans=SL[EnvelopingSpan(['isuäratavad'], [{'pattern': 'MSEQ(A)', 'priority': 2}]),
EnvelopingSpan(['isuäratavad', 'ja', 'maitsvad'], [{'pattern': 'A J A', 'priority': 1}]),
EnvelopingSpan(['maitsvad', 'saavad'], [{'pattern': 'MSEQ(A)', 'priority': 2}]),
EnvelopingSpan(['häid', 'mahlaseid'], [{'pattern': 'MSEQ(A)', 'priority': 2}]),
EnvelopingSpan(['hapukatest'], [{'pattern': 'MSEQ(A)', 'priority': 2}])])

Tulemus on sama, mis `grammar_2` kasutades, kuid nüüd tuleb kasuks dekoraatoris tekitatud atribuut *priority*. 
Kasutame sedasama `resolve_conflicts`-meetodit, mida mäletatavasti kasutas ka PhraseTagger oma "kõhus", et kattuvusi lahendada. Nii saame dekoraatoris tekitatud parameetrite kaudu konfliktidest jagu:

In [30]:
from estnltk.layer_operations import resolve_conflicts

# Viimaseks lahendame konfliktid meetodiga resolve_conflicts
resolve_conflicts(text.adjectives_3,  # tekstikiht, mida lahendatakse                 
                  conflict_resolving_strategy='ALL',
                  priority_attribute='priority',
                  status=None,
                  keep_equal=True )

Layer(name='adjectives_3', attributes=('pattern', 'priority'), spans=SL[EnvelopingSpan(['isuäratavad', 'ja', 'maitsvad'], [{'pattern': 'A J A', 'priority': 1}]),
EnvelopingSpan(['häid', 'mahlaseid'], [{'pattern': 'MSEQ(A)', 'priority': 2}]),
EnvelopingSpan(['hapukatest'], [{'pattern': 'MSEQ(A)', 'priority': 2}])])

Uuesti kihti vaadates näeme, et üleliigsed vasted on nüüd tõepoolest kaotatud reegli (A J A) kasuks.